<a href="https://colab.research.google.com/github/alickchoi/STAT5106-Lab/blob/main/Week_4_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initiation of a DataFrame

idea: [Hong Kong Airport flight schedule](https://www.hongkongairport.com/en/flights/departures/passenger.page)

Further Reference: [The API](https://data.gov.hk/tc-data/dataset/aahk-team1-flight-info)

Feel free to try

```python
import requests
import pandas as pd
import numpy as np

url = 'https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=2025-09-19&lang=en&cargo=false&arrival=false'
json_data = requests.get(url).json()

df = pd.json_normalize(json_data[1], record_path=['list'], meta = ['date']).explode('flight').explode('destination').reset_index(drop=True)
df_flight = pd.json_normalize(df.flight)
df = pd.concat([df.drop('flight', axis=1), df_flight], axis=1)

np.random.seed(9527)
df.sample(3)
```



In [ ]:
import pandas as pd

In [ ]:
# How to build a simple data frame
flights = [
    {'date': '2025-09-19 00:05', 'flight no': 'BA 4149', 'destination':'Perth', 'Check-in': 'BC'}
    , {'date': '2025-09-19 07:15', 'flight no': 'UO 618', 'destination':'Seoul', 'Check-in': 'H'}
    , {'date': '2025-09-19 14:30', 'flight no': 'PG 4558', 'destination':'Bangkok', 'Check-in': 'A'}
]
df_flights = pd.DataFrame(flights)
print(df_flights)

In [ ]:
# How to build a simple data frame, by columns
flights_col = {
    'data no': [1, 2, 3]
    , 'date': ['2025-09-19 00:05', '2025-09-19 07:15', '2025-09-19 14:30']
    , 'flight no': ['BA 4149', 'UO 618', 'PG 4558']
}
df_flights_col = pd.DataFrame(flights_col)
print(df_flights_col)

In [ ]:
# build data frame with data number as Index
df_flights = pd.DataFrame(flights, index=[100, 101, 102])
print(df_flights)

In [ ]:
df_2 = pd.DataFrame(flights)
df_2 = df_2.set_index('flight no')

In [ ]:
df_2.loc['PG 4558']

In [ ]:
# use loc for selecting data
print(df_flights.loc[102])

In [ ]:
print(df_flights.iloc[2])

In [ ]:
df_flights

In [ ]:
print(df_flights[['flight no', 'destination']])

In [ ]:
# using loc and iloc for selecting a specific data elements
print(df_flights.loc[101, 'date'])
print(df_flights.iloc[1,0])

# MovieLens

* [Data Source Link](https://grouplens.org/datasets/movielens/)
* The small dataset version is involved – [ml-latest-small.zip](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip), with 100K ratings <br/>
(Of course you can play the larger version if you are interested in)
* 4 files are consisted ([details](http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html))

In [ ]:
# Download file

from urllib.request import urlretrieve
urlretrieve("http://files.grouplens.org/datasets/movielens/ml-latest-small.zip", "ml-latest-small.zip")

In [ ]:
# Directly read csvs in Zip file

import zipfile

zf = zipfile.ZipFile('ml-latest-small.zip')
df_movies = pd.read_csv(zf.open('ml-latest-small/movies.csv'), dtype='str')
df_ratings = pd.read_csv(zf.open('ml-latest-small/ratings.csv'), dtype='str')

In [ ]:
# 1st step of analysis, review data frame structure.
print(df_movies.info(), "\n")
print(df_movies.describe(), "\n")
print(df_movies.head())
print(df_movies.tail())

In [ ]:
# get df row and columns
print(df_movies.shape)

In [ ]:
# when you used the zip handler, please close it.
zf.close()

In [ ]:
# count the unique number of movies
df_movies['movieId'].nunique()

In [ ]:
df_ratings.head()

In [ ]:
df = df_ratings.merge(df_movies, how='left', on='movieId')
print(df.tail())

In [ ]:
#Find the peak month users rate movies
df['datetime'] = pd.to_datetime(df.timestamp, unit='s')
df['months'] = df['datetime'].dt.month

In [ ]:
df.head()

In [ ]:
df['months'].value_counts()

In [ ]:
#Find the peak hour users rate movies
df['hours'] = df.datetime.dt.hour
print(df['hours'].value_counts().index.values[0], 'th hour is the peak hour')

In [ ]:
df['hours'].value_counts().index[0]

In [ ]:
import seaborn as sns
sns.countplot(data = df, x = 'hours', palette='Purples')

# Chunk in read_csv

In [ ]:
import zipfile

zf = zipfile.ZipFile('ml-latest-small.zip')

In [ ]:
iter_df_movies = pd.read_csv(zf.open('ml-latest-small/movies.csv'), chunksize=1024)

In [ ]:
type(iter_df_movies)

In [ ]:
# iterate for each small chunk
df_movies_mini = next(iter_df_movies)
print(df_movies_mini.head(1), df_movies_mini.tail(1))

In [ ]:
# iterate for each small chunk
df_movies_mini = next(iter_df_movies)
print(df_movies_mini.head(1), df_movies_mini.tail(1))

In [ ]:
# for loops
for df_movies_mini in iter_df_movies:
    print("Reading rows", df_movies_mini.index[0], df_movies_mini.index[-1])

In [ ]:
next(iter_df_movies)

# Generators

In [ ]:
def gen_example():
    n = 1
    print('This is printed first')
    # Generator function contains yield statements
    yield n

    n += 1
    print('This is printed second')
    yield n

    n += 1
    print('This is printed at last')
    yield n

In [ ]:
g_eg = gen_example()
print(g_eg)

In [ ]:
print(next(g_eg))

In [ ]:
def processing_rows(zipfilename, csvfile):
    zf_gen = zipfile.ZipFile(zipfilename)
    iter_df_movies = pd.read_csv(zf.open(csvfile), chunksize=1024)
    for df_movies_mini in iter_df_movies:
        yield (df_movies_mini.index[0], df_movies_mini.index[-1])

g_df = processing_rows('ml-latest-small.zip', 'ml-latest-small/movies.csv')

In [ ]:
next(g_df)

In [ ]:
for g_df_out in g_df:
    print("Reading rows", g_df_out)